In [6]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import nbinteract as nbi
import pandas as pd
from functools import lru_cache
import requests

@lru_cache(maxsize=128)
def get_airport_info(code):
    import requests
    
    import datetime
    today = datetime.date.today()
    first = today.replace(day=1)
    lastMonthLastDay = first - datetime.timedelta(days=1)
    toDate = lastMonthLastDay.strftime('%m/%d/%Y')
    lastMonthFirstDay = lastMonthLastDay.replace(day=1)
    fromDate = lastMonthFirstDay.strftime('%m/%d/%Y')
    
    url = "https://awt.cbp.gov"
    data = {
        "port": code,
        "rptFrom": fromDate,
        "rptTo": toDate
    }

    res = requests.post(url, data)
    soup = BeautifulSoup(res.content)
    table = soup.find_all('table')[0] 
    dfs = pd.read_html(str(table))
    df = dfs[0]
    df.columns = [' '.join(list(dict.fromkeys(col))).strip() for col in df.columns.values]
    # most of our analysis will be around day of
    
    # convert date and hour to a time datatype
    l = [d.split('--')[0] + ' ' + d.split('--')[1].split('-')[0][:2] for d in df['Date'] + '--' + df['Hour']]
    df['ts'] = pd.to_datetime(pd.Series(l), format="%m/%d/%Y %H")
    df = df.set_index('ts')
    df_hour_of_day_mean = df.groupby(df.index.hour).mean()
    return df_hour_of_day_mean

def y_f(xs, code):
    df = get_airport_info(code)
    return df['U.S. Citizen Max Wait  Time']
    
def x_f():
    return range(0,24)

opts = {
    'xlim': (0, 24),
}

nbi.bar(x_f, y_f, code=['SFO', 'JFK'], options=opts)